In [ ]:
!pip install keras_facenet

     |████████████████████████████████| 2.3MB 4.1MB/s 
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-cp36-none-any.whl size=10387 sha256=ec9bf2f4d5b21accdf0bddb9386c76f13cd868731dafad118c53a22134df9c0d
  Stored in directory: /root/.cache/pip/wheels/f6/53/9a/36c4b52fd22faf4f710d5047d874655b85a1b2cf77accfb9bd
Successfully built keras-facenet


In [ ]:
from os.path import join, exists
from os import listdir, makedirs
import sys
from matplotlib import pyplot as plt
import numpy as np
import time
import os
import h5py
import cv2

from keras_facenet import FaceNet
from keras.layers.pooling import GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from keras import backend as K
from keras import applications, Sequential, optimizers
from keras.layers import Flatten, Dropout, Dense
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r --recursive "/content/drive/My Drive/data.zip" "/content"
!unzip data.zip

In [ ]:
trainX , testX , valX = [] , [] , []
trainY , testY , valY = [] , [] , []
counter = 0
facenet = FaceNet()

for dataset in ["train","test","val"]:
    for ind,label in enumerate(["manipulated_sequences","original_sequences"]):
        data_path = os.path.join("data",dataset,label)
        for img_name in os.listdir(data_path):
            img_path = os.path.join(data_path, img_name)

            img = cv2.imread(img_path)
            # print(img.shape)

            img = np.expand_dims(img, axis=0)
            # print(img.shape)

            embeddings = facenet.embeddings(img)
            # print(embeddings.shape)

            if dataset == "train":
                trainX.append(np.transpose(embeddings).flatten())
                trainY.append(ind)
            elif dataset == "test":
                testX.append(np.transpose(embeddings).flatten())
                testY.append(ind)
            else:
                valX.append(np.transpose(embeddings).flatten())
                valY.append(ind)
            
            counter += 1 

            sys.stdout.write('\r')
            sys.stdout.write(str(counter))
            sys.stdout.flush()

In [ ]:
trainX = np.array(trainX)
trainY = np.array(trainY)
trainY = trainY.reshape((len(trainY),1))

testX = np.array(testX)
testY = np.array(testY)
testY = testY.reshape((len(testY),1))

valX = np.array(valX)
valY = np.array(valY)
valY = valY.reshape((len(valY),1))


print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)
print(valX.shape)
print(valY.shape)

In [ ]:
archive = h5py.File('train160.h5', 'w')
archive.create_dataset('/X', data = trainX)
archive.create_dataset('/Y',data = trainY)
archive.close()

archive = h5py.File('test160.h5', 'w')
archive.create_dataset('/X', data = testX)
archive.create_dataset('/Y',data = testY)
archive.close()

archive = h5py.File('val160.h5', 'w')
archive.create_dataset('/X', data = valX)
archive.create_dataset('/Y',data = valY)
archive.close()

In [ ]:
!cp -r --recursive  "train160.h5"        "/content/drive/My Drive"
!cp -r --recursive  "test160.h5"        "/content/drive/My Drive"
!cp -r --recursive  "val160.h5"        "/content/drive/My Drive"

In [ ]:
dataset = h5py.File('train160.h5', "r")
x = dataset["X"][:]
y = dataset["Y"][:]
print(x.shape,y.shape)
dataset.close()


dataset = h5py.File('test160.h5', "r")
x = dataset["X"][:]
y = dataset["Y"][:]
print(x.shape,y.shape)
dataset.close()


dataset = h5py.File('val160.h5', "r")
x = dataset["X"][:]
y = dataset["Y"][:]
print(x.shape,y.shape)
dataset.close()